In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from tqdm.auto import tqdm
import torch

In [ ]:
train_df = pd.read_csv("./data/train.csv")
train_df.head()

,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),traffic(t),type
0,3.0,2.0,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,0
1,2.0,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,0
2,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,0
3,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,6.0,0
4,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,6.0,17.0,0


In [ ]:
from itertools import combinations
combi_list = sorted(combinations(np.arange(11),2))
for combin in list(combi_list):
    i,j = list(combin)[0],list(combin)[1]
    train_df[f'new_feature{i}_{j}'] = train_df.iloc[:,i]/train_df.iloc[:,j]

# Split and scale train data into stratified 5 folds

In [ ]:
X,y = train_df.drop('type',axis = 1).to_numpy(),train_df['type']

In [ ]:
X_scaled = (X-X.mean())/X.std()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,random_state=38,shuffle = True)
skf.get_n_splits(X_scaled, y)

5

In [ ]:
global fold_dict
fold_dict = dict()

for i, (train_index, test_index) in enumerate(skf.split(X_scaled,y)):
    fold_dict[f'fold{i}'] = {'train_indices':train_index, 'test_indices':test_index}

In [ ]:
fold_dict.keys()

dict_keys(['fold0', 'fold1', 'fold2', 'fold3', 'fold4'])

In [ ]:
from collections import Counter
print(Counter(y[fold_dict['fold0']['test_indices']]))
print(Counter(y[fold_dict['fold1']['test_indices']]))
print(Counter(y[fold_dict['fold2']['test_indices']]))
print(Counter(y[fold_dict['fold3']['test_indices']]))
print(Counter(y[fold_dict['fold4']['test_indices']]))

Counter({0: 1578, 1: 588, 2: 182, 3: 181})
Counter({0: 1577, 1: 589, 2: 182, 3: 181})
Counter({0: 1577, 1: 589, 3: 182, 2: 181})
Counter({0: 1577, 1: 589, 3: 182, 2: 181})
Counter({0: 1577, 1: 588, 2: 182, 3: 182})


# Model

In [ ]:
class SVMClassifier:
    def __init__(self,n_iters=100, lr = 0.0001, random_seed=3, lambda_param=0.01):
        self.n_iters = n_iters # 몇 회 반복하여 적절한 값을 찾을지 정하는 파라미터
        self.lr = lr  # 학습률과 관련된 파라미터 
        self.lambda_param = lambda_param
        self.random_seed = random_seed
        np.random.seed(self.random_seed)


    def fit(self, x_, y_orig):
        """
        본 함수는 x, y를 활용하여 훈련하는 과정을 코딩하는 부분입니다.
        아래 reference 사이트의 gradient 계산 부분을 참고하세요.
        reference: https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47
        아래 총 6개의 None 부분을 채우시면 됩니다.

        """
        n_samples, n_features = x_.shape

        # hint: y값을 SVM 계산에 활용해주기 위하여 0에 해당하는 y값들을 -1로 변환
        y_ =  np.array(y_orig)#numpy array의 y를 y_에 담아줍니다
        y_[y_ == 0] = -1#0에 해당하는 y_값들을 -1로 변환합니다.
        
        # hint: w값 초기화, (n_features, )의 크기를 가지는 0과 1사이의 랜덤한 변수 어레이 (필수: 넘파이로 정의해야 함)
        init_w = np.random.rand(n_features)#0과 1 사이의 랜덤한 수로 init_w 배열을 초기화합니다
        self.w = init_w
        self.b = 0 # b값 초기화
        n_label = np.sum((y_==1)*1)
        
        for _ in range(self.n_iters):
            non_label_indices = np.where(y_ == -1)[0]
            label_indices = np.where(y_ != -1)[0]
            #print("# non_label_indices:",len(non_label_indices))
            #print("# label_indices:",len(label_indices))
            
            more_sample_n = len(non_label_indices)-len(label_indices)
            #print("# more sampleing number:",more_sample_n)
            n_iter = _
            if more_sample_n <0:
                more_sample_indices = np.random.choice(non_label_indices,size = -more_sample_n,replace = False)
            else:
                more_sample_indices = np.random.choice(label_indices,size = more_sample_n,replace = True)
            x = np.vstack([x_,x_[more_sample_indices,:]])
            y = np.concatenate([y_,y_[more_sample_indices]])
            assert x.shape[0] == y.shape[0]
            for i in range(n_samples):
                x_i = x[i]
                y_i = y[i]

                # hint: y(i) * (w · x(i) + b) >= 1 를 만족하는 경우의 의미가 담기도록 if문을 채우세요.
                condition = ((y_i*np.dot(self.w,x_i)+self.b)>=1) #y(i) * (w · x(i) + b) >= 1 를 만족하는 경우를 boolean으로 표현한 것으로, 조건이 부합하면 True, 부합하지 않으면 False를 condition에 담습니다
                if condition:#condition이 True일 때 
                    # hint: w에 대하여 Gradient Loss Function 수식을 이용하여 W를 업데이트 하세요.
                    self.w -= self.lr * 2*self.lambda_param*self.w#misclassification이 없는 경우 가중치 갱신 공식: w = w-lr*2*lambda*w
                else:#condition이 False일 때
                    # hint: w에 대하여 Gradient Loss Function 수식을 이용하여 W를 업데이트 하세요.
                    self.w -= self.lr * (-y_i*x_i+2*self.lambda_param*self.w)#misclassification인 경우 가중치 갱신 공식: w = w + lr*(y_i*x_i-2*lambda*w)
                    self.b -= self.lr * y_i

        return self.w, self.b #w와 b를 반환합니다


    def predict(self, x):
        """
            [n_samples x features]로 구성된 x가 주어졌을 때, fit을 통해 계산된 
            self.w와 self.b를 활용하여 예측값을 계산합니다.

            @args:
                [n_samples x features]의 shape으로 구성된 x
            @returns:
                [n_samples, ]의 shape으로 구성된 예측값 array

            아래의 수식과 수도코드를 참고하여 함수를 완성하면 됩니다.
                approximation = W·X - b
                if approximation >= 0 {
                    output = 1
                }
                else{
                    output = 0
                }
        """
        approx = np.matmul(x,self.w)-self.b#W*X-b를 vectorization으로 계산합니다
        approx[approx>=0] = 1#approximation이 0 이상인 경우 1로 예측하고
        approx[approx<0] = 0#approximation이 0 미만인 경우 0으로 예측합니다
        return approx#예측 값을 담은 배열인 approx를 반환합니다
    
    def sigmoid(self,z):
        return 1./(1.+np.exp(-z))
    
    def get_prob(self,X):
        approx = np.matmul(x,self.w)-self.b
        return self.sigmoid(approx)


    def get_accuracy(self, y_true, y_pred):
        """
            y_true, y_pred가 들어왔을 때, 정확도를 계산하는 함수.
            sklearn의 accuracy_score 사용 불가능 / sklearn의 accuracy_score 함수를 구현한다고 생각하면 됩니다.
            넘파이만을 활용하여 정확도 계산 함수를 작성하세요.
        """
        acc = np.sum(y_true == y_pred)/len(y_pred)#예측하고자 하는 전체 데이터 수 개수(len(y_pred)) 중 예측 값과 실제 값이 일치하는 데이터 개수(np.sum(y_true == y_pred))의 비율로 정확도를 계산합니다
        return acc #정확도를 담은 acc를 반환합니다

# Utils

In [ ]:
def train_test_binary_model(model,label):
    avg_train_acc = []
    avg_test_acc = []
    for fold in tqdm(fold_dict.values(),leave=False,desc = "5 folds..."):
        train_X,train_y = X_scaled[fold['train_indices']],y[fold['train_indices']]
        test_X,test_y = X_scaled[fold['test_indices']],y[fold['test_indices']]
        use_train_y,use_test_y = (train_y==label)*1,(test_y==label)*1
        model.fit(train_X,use_train_y)
        train_preds = model.predict(train_X)
        train_acc = np.mean(train_preds == use_train_y)
        test_preds = model.predict(test_X)
        test_acc = np.mean(test_preds == use_test_y)
        avg_train_acc.append(train_acc)
        avg_test_acc.append(test_acc)
    return np.mean(np.array(avg_train_acc)),np.mean(np.array(avg_test_acc))

In [ ]:
import copy
import queue
pqueue = queue.PriorityQueue()
lrs = [0.1,0.01,0.005,0.001,0.0005,0.0001]
C = [0.001,0.01,0.1,1.,5.]
best_param0 = dict()
best_acc_0 = 0
best_train_acc = 0
save_models = dict()
for C_ in C:
    for lr in lrs:
        svm0 = SVMClassifier(lambda_param=C_,lr=lr)
        train_acc,test_acc = train_test_binary_model(svm0,2)
        print(f"(lr:{lr}, C:{C_} label:0) acc: {round(train_acc,4)} val acc:{round(test_acc,3)}")
        if train_acc > best_train_acc and test_acc>=best_acc_0:
            print(">>best model updated\n")
            best_acc_0 = test_acc
            best_param0['lr'] = lr
            best_param0['lambda']=C_
            best_svm0 = copy.deepcopy(svm0)
            best_train_acc = train_acc
            save_name = f"./weights/div_oversampling_label2_{str(best_acc_0)[:4]}.pt"
            best_param0['train_acc'] = best_train_acc
            best_param0['test_acc'] = best_acc_0
            best_param0['best_model'] = best_svm0
            torch.save(best_param0,save_name)
        if test_acc > 0.85:
            save_models[test_acc] = {'train_acc':train_acc,'model':copy.deepcopy(svm0)}

5 folds...:   0%|          | 0/5 [00:00<?, ?it/s]

(lr:0.1, C:0.001 label:0) acc: 0.8411 val acc:0.84
>>best model updated



5 folds...:   0%|          | 0/5 [00:00<?, ?it/s]

(lr:0.01, C:0.001 label:0) acc: 0.8242 val acc:0.825


5 folds...:   0%|          | 0/5 [00:00<?, ?it/s]

...

(lr:0.001, C:5.0 label:0) acc: 0.9282 val acc:0.928


5 folds...:   0%|          | 0/5 [00:00<?, ?it/s]

(lr:0.0005, C:5.0 label:0) acc: 0.9282 val acc:0.928


5 folds...:   0%|          | 0/5 [00:00<?, ?it/s]

(lr:0.0001, C:5.0 label:0) acc: 0.9282 val acc:0.928


In [ ]:
print("best_train_acc:",best_train_acc,"best test acc:",best_acc_0)
best_param0

best_train_acc: 0.9373270067220243 best test acc: 0.9361803084223013


{'lr': 0.1,
 'lambda': 0.01,
 'train_acc': 0.9373270067220243,
 'test_acc': 0.9361803084223013,
 'best_model': <__main__.SVMClassifier at 0x1ae6de76f70>}

In [ ]:
import torch
save_name = f"./weights/div_oversampling_label2_{str(best_acc_0)[:4]}.pt"
best_param0['train_acc'] = best_train_acc
best_param0['test_acc'] = best_acc_0
best_param0['best_model'] = best_svm0
torch.save(best_param0,save_name)

In [ ]:
save_models

{0.9361803084223013: {'train_acc': 0.9373270067220243,
  'model': <__main__.SVMClassifier at 0x1ae6de76d90>},
 0.8631079478054566: {'train_acc': 0.8626136812969554,
  'model': <__main__.SVMClassifier at 0x1ae6de4f9a0>},
 0.8578094108343219: {'train_acc': 0.8566231712139187,
  'model': <__main__.SVMClassifier at 0x1ae6de8e070>},
 0.9281929616449188: {'train_acc': 0.928192961644919,
  'model': <__main__.SVMClassifier at 0x1ae6df846d0>},
 0.9284302095689997: {'train_acc': 0.9285686041913799,
  'model': <__main__.SVMClassifier at 0x1ae6dead0a0>},
 0.9288256227758007: {'train_acc': 0.9287465401344404,
  'model': <__main__.SVMClassifier at 0x1ae6dedbf70>},
 0.9266903914590747: {'train_acc': 0.925721629102412,
  'model': <__main__.SVMClassifier at 0x1ae6dec2ee0>},
 0.9268485567417951: {'train_acc': 0.9265322261763543,
  'model': <__main__.SVMClassifier at 0x1ae6deedaf0>}}

In [ ]:
use_keys = sorted(list(save_models.keys()),reverse = True)[:4]
use_keys

[0.9361803084223013,
 0.9288256227758007,
 0.9284302095689997,
 0.9281929616449188]

# Predict with all folds

In [ ]:
save_models

{0.9361803084223013: {'train_acc': 0.9373270067220243,
  'model': <__main__.SVMClassifier at 0x1ae6de76d90>},
 0.8631079478054566: {'train_acc': 0.8626136812969554,
  'model': <__main__.SVMClassifier at 0x1ae6de4f9a0>},
 0.8578094108343219: {'train_acc': 0.8566231712139187,
  'model': <__main__.SVMClassifier at 0x1ae6de8e070>},
 0.9281929616449188: {'train_acc': 0.928192961644919,
  'model': <__main__.SVMClassifier at 0x1ae6df846d0>},
 0.9284302095689997: {'train_acc': 0.9285686041913799,
  'model': <__main__.SVMClassifier at 0x1ae6dead0a0>},
 0.9288256227758007: {'train_acc': 0.9287465401344404,
  'model': <__main__.SVMClassifier at 0x1ae6dedbf70>},
 0.9266903914590747: {'train_acc': 0.925721629102412,
  'model': <__main__.SVMClassifier at 0x1ae6dec2ee0>},
 0.9268485567417951: {'train_acc': 0.9265322261763543,
  'model': <__main__.SVMClassifier at 0x1ae6deedaf0>}}

In [ ]:
import copy
train_all_folds_models = copy.deepcopy(save_models)
train_all_folds_models

{0.9361803084223013: {'train_acc': 0.9373270067220243,
  'model': <__main__.SVMClassifier at 0x1ae6f3c1040>},
 0.8631079478054566: {'train_acc': 0.8626136812969554,
  'model': <__main__.SVMClassifier at 0x1ae6f7b9940>},
 0.8578094108343219: {'train_acc': 0.8566231712139187,
  'model': <__main__.SVMClassifier at 0x1ae6f7b9460>},
 0.9281929616449188: {'train_acc': 0.928192961644919,
  'model': <__main__.SVMClassifier at 0x1ae6f7b91f0>},
 0.9284302095689997: {'train_acc': 0.9285686041913799,
  'model': <__main__.SVMClassifier at 0x1ae6f7b9c10>},
 0.9288256227758007: {'train_acc': 0.9287465401344404,
  'model': <__main__.SVMClassifier at 0x1ae6f7b9640>},
 0.9266903914590747: {'train_acc': 0.925721629102412,
  'model': <__main__.SVMClassifier at 0x1ae6f3bf9a0>},
 0.9268485567417951: {'train_acc': 0.9265322261763543,
  'model': <__main__.SVMClassifier at 0x1ae6f3bf0d0>}}

In [ ]:
all_folds_models = sorted(list(train_all_folds_models.keys()),reverse = True)[:5]
all_folds_models

[0.9361803084223013,
 0.9288256227758007,
 0.9284302095689997,
 0.9281929616449188,
 0.9268485567417951]

In [ ]:
label = 2
for model_key in all_folds_models:
    print(f"fitting and predicting with:{model_key}")
    train_all_folds_models[model_key]['model'].fit(X_scaled, (y==label)*1)
    train_acc = np.mean(train_all_folds_models[model_key]['model'].predict(X_scaled)==(y==label)*1)
    print(f"train acc:{train_acc}")
    final_preds = np.zeros((X_test.shape[0],4))
    preds = train_all_folds_models[model_key]['model'].predict(scaled_X_test)
    final_preds[:,label] = preds*model_key
    torch.save({f"prediction{label}":final_preds},f"./predictions/all_folds_div_label{label}_{str(model_key)}.pt")

fitting and predicting with:0.9361803084223013
train acc:0.9371293001186239
fitting and predicting with:0.9288256227758007
train acc:0.9284302095689996
fitting and predicting with:0.9284302095689997
train acc:0.9284302095689996
fitting and predicting with:0.9281929616449188
train acc:0.9281929616449189
fitting and predicting with:0.9268485567417951
train acc:0.9264531435349941
